In [2]:
include("src//sgs.jl")

using .sgs
using JuMP

In [3]:
sgs.greeting()

Hello World!


In [4]:
animals = ["dog", "cat", "chicken", "cow", "pig"]
model = Model()
@variable(model, x[animals])

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, ["dog", "cat", "chicken", "cow", "pig"]
And data, a 5-element Vector{VariableRef}:
 x[dog]
 x[cat]
 x[chicken]
 x[cow]
 x[pig]

In [5]:
weight_of_animals = Dict(
    "dog" => 20.0,
    "cat" => 5.0,
    "chicken" => 2.0,
    "cow" => 720.0,
    "pig" => 150.0,
)
animal_keys = keys(weight_of_animals)
model = Model()
@variable(model, x[animal_keys])

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, ["cow", "chicken", "cat", "pig", "dog"]
And data, a 5-element Vector{VariableRef}:
 x[cow]
 x[chicken]
 x[cat]
 x[pig]
 x[dog]

In [6]:
animal_set = Set()
for animal in keys(weight_of_animals)
    push!(animal_set, animal)
end
animal_set

Set{Any} with 5 elements:
  "cow"
  "chicken"
  "cat"
  "pig"
  "dog"

In [7]:
push!(animal_set, "dog")
animal_set

Set{Any} with 5 elements:
  "cow"
  "chicken"
  "cat"
  "pig"
  "dog"

In [8]:
model = Model()
@variable(model, x[animal_set])

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, ["cow", "chicken", "cat", "pig", "dog"]
And data, a 5-element Vector{VariableRef}:
 x[cow]
 x[chicken]
 x[cat]
 x[pig]
 x[dog]

In [9]:
sources = ["A", "B", "C"]
sinks = ["D", "E"]
S = [(source, sink) for source in sources, sink in sinks]
model = Model()
@variable(model, x[S])

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [("A", "D"), ("B", "D"), ("C", "D"), ("A", "E"), ("B", "E"), ("C", "E")]
And data, a 6-element Vector{VariableRef}:
 x[("A", "D")]
 x[("B", "D")]
 x[("C", "D")]
 x[("A", "E")]
 x[("B", "E")]
 x[("C", "E")]

In [ ]:
x[("A", "D")]

In [10]:
# Example Vector{UInt64}
vec_uint64 = Float64[0, 2^3, 2^15, 2^54]

# Convert Vector{UInt64} to Vector{Float16}
vec_float16 = Float16.(vec_uint64)

println(vec_float16)

# Convert Vector{Float16} to bytes
bytes = reinterpret(UInt8, vec_float16)

println(bytes)

# If you need a flat Vector{UInt8} instead of a matrix
flat_bytes = vec(bytes)

println(flat_bytes)

println(Int16(2^11 - 1))


Float16[0.0, 8.0, 3.277e4, Inf]
UInt8[0x00, 0x00, 0x00, 0x48, 0x00, 0x78, 0x00, 0x7c]
UInt8[0x00, 0x00, 0x00, 0x48, 0x00, 0x78, 0x00, 0x7c]
2047


In [11]:
struct MyStruct
    original_value::Float64
    converted_value::Float16

    # Inner constructor
    function MyStruct(original_value::Float64)
        new(original_value, Float16(original_value))
    end
end

# Example usage
obj = MyStruct(123.456)
println(obj.original_value)  # Prints: 123.456
println(obj.converted_value) # Prints: Float16(123.5)

123.456
123.44


In [ ]:
using Redis 

conn = Redis.RedisConnection()
Redis.set(conn, "foo", "bard")
println(Redis.get(conn, "foo"))
println(typeof(conn))


# inclose the following code in try catch block
try
    # Create an index with RediSearch
    """
        FT.CREATE edges 
            ON HASH 
            PREFIX 1 b:e
        SCHEMA
            author_id TAG SORTABLE 
            author_ids TAG 
            title TEXT name TEXT SORTABLE
    """
    Redis.execute_command(conn, ["FT.CREATE", "edges", "ON", "HASH", "PREFIX", 1, "b:e:", 
    "SCHEMA", "source", "TEXT", "target", "TEXT", "r_type", "TEXT", "props", "TEXT"])
catch e
    println(e)
end

# try
#     # Add a document to the index
#     Redis.hmset(conn, "token:doc1", Dict(:title => "Hello World", :body => "Lorem ipsum dolor sit amet")) #"1.0", "FIELDS", ])
# catch e
#     println(e)
# end

# Search the index
search_result = Redis.execute_command(conn, ["FT.SEARCH", "edges", "*"])

println(search_result)


# Redis.disconnect(conn)

In [ ]:
# Your byte array in Julia
byte_array = Vector{UInt8}([0x01, 0x02, 0xFF])

# Convert byte array to string and store in Redis
Redis.set(conn, "byte_key", byte_array)

# Retrieve the string from Redis and convert back to byte array
retrieved_bytes = Vector{UInt8}(Redis.get(conn, "byte_key"))

println(retrieved_bytes)  # Should print the original byte_array content

In [4]:
using Redis
conn = Redis.RedisConnection()
try
    # Create an index with RediSearch    
    Redis.execute_command(conn, ["FT.CREATE", "tokens", "ON", "HASH", "PREFIX", 1, "b:token:", 
        "SCHEMA", "id", "NUMERIC", "bin", "NUMERIC", "zeros", "NUMERIC", "tf", "NUMERIC", "token", "TEXT", "searchable",  
        "VECTOR", 
        "FLAT", 
        "10", 
        "TYPE", 
        "FLOAT32", 
        "DIM", 
        "256", 
        "DISTANCE_METRIC", 
        "IP", 
        "INITIAL_CAP", "1000000", 
        "BLOCK_SIZE", "1000"])

    Redis.execute_command(conn, ["FT.CREATE", "nodes", "ON", "HASH", "PREFIX", 1, "b:n:", 
        "SCHEMA", "sha1", "TEXT", "labels", "TEXT", "searchable", 
        "VECTOR", 
        "HNSW", 
        "16", 
        "TYPE", 
        "FLOAT32", 
        "DIM", 
        "1024", 
        "DISTANCE_METRIC", 
        "IP",
        "INITIAL_CAP", 
        "1400000", 
        "M", 
        "40", 
        "EF_CONSTRUCTION", 
        "100", 
        "EF_RUNTIME", 
        "20", 
        "EPSILON", 
        "0.8"])
catch e
    println(e)
end

"OK"

In [2]:
using Redis
conn = Redis.RedisConnection()

search_result = Redis.execute_command(conn, ["FT.SEARCH", "nodes", "*=>[KNN 10 @search $BLOB]", "PARAMS", "2", "BLOB", "\x12\xa9\xf5\x6c", "DIALECT", "4"])

println(search_result)



LoadError: UndefVarError: `BLOB` not defined

In [2]:
include("src//sets.jl")

import .HllSets as hset

Redis.sadd(conn, "test", Dict(:a => "a1", :b => "a2", :c => "a3"))
result = Redis.smembers(conn, "test")

hll = hset.HllSet{8}()
hset.add!(hll, result)


In [7]:
include("src//utils.jl")

import .Util as util
dump = hset.dump(hll)

str_ = util.to_blob(dump)

# util.from_blob(str_)

to_blob: 256; 1024


128-element reinterpret(UInt64, ::Vector{UInt8}):
 0x3030303030303030
 0x3030303030303030
 0x3030303030303030
 0x3030303030303030
 0x3030303030303030
 0x3030303030303030
 0x3030303030303030
 0x3030303030303030
 0x3030303030303030
 0x3030303030303030
 0x3030303031303030
 0x3030303030303030
 0x3030303030303030
                  ⋮
 0x3030303030303030
 0x3030303030303030
 0x3030303030303030
 0x3030303030303030
 0x3030303030303030
 0x3030303030303030
 0x3030303032303030
 0x3030303030303030
 0x3130303030303030
 0x3030303030303030
 0x3030303030303030
 0x3030303030303030

1. https://support.xilinx.com/s/question/0D54U0000889tzVSAQ/vitis-unified-getting-started-and-featured-tutorials?language=en_US
2. https://github.com/Xilinx/Vitis-Tutorials/tree/2023.2
3. https://pretalx.com/juliacon2024/talk/RJEPZY/
4. https://en.cs.uni-paderborn.de/fileadmin-eim/informatik/fg/hpc/teaching/thesis-topics/2023-08-31-XRT-for-julia.pdf 
5. https://xilinx.github.io/XRT/master/html/
6. https://towardsdatascience.com/train-naive-bayes-really-fast-7398a404e342
7. https://compositionality-journal.org/papers/compositionality-4-5/
8. https://github.com/topics/compositionality 
9. http://www.tac.mta.ca/tac/volumes/32/16/32-16.pdf (algebraic databases)
10. https://duckdb.org/docs/api/julia.html
11. https://medium.com/@rubenszimbres/kolmogorov-arnold-networks-a-critique-2b37fea2112e 
12. https://www.categoricaldata.net/fql.html 
13. https://conexus.com/
14. https://zoumanakeita.medium.com/multimodal-retrieval-augmented-generation-for-sustainable-finance-1bc664ac2511 
15. https://towardsdatascience.com/understanding-conditional-probability-and-bayes-theorem-b779f8801ef6  
16. https://medium.com/coinmonks/implementing-a-mersenne-twister-generator-in-rust-634c7ff93b30

Tools

1. https://levelup.gitconnected.com/10-examples-why-curl-is-an-awesome-cli-tool-0b23223ec845

Philosophy

1.  https://archive.org/details/DaoDeJing/page/n3/mode/2up
2.  https://bilinguator.com/ru/online?book=1228&page=1
3.  https://www.chinasage.info/dao-de-jing.htm
4.  https://www.yellowbridge.com/onlinelit/daodejing01.php
5.  https://www.teemuqi.org/single-post/2016/08/07/guodian-laozi-and-the-textual-history-of-the-daode-jing
6.  https://www.zhongwen.com/dao.htm
7.  http://www.sanmayce.com
8.  https://msrabota.ru/content/book_docs/lao-tszi_dao_de_tszin1.pdf1_.pdf